In [369]:
import numpy as np
import os
from scipy.linalg import sqrtm

In [370]:
def getCB(B):
    """
    Compute the CB matrix from the B matrix.
    (1 + i B)(1 - iB)^{-1}
    """
    I = np.eye(B.shape[0], dtype=complex)
    return (I + 1j * B) @ np.linalg.inv(I - 1j * B)

def getStilde(Ktilde):
    """
    Compute the Stilde matrix from the Ktilde matrix.
    (1 + i Ktilde)(1 - i Ktilde)^{-1}
    """
    I = np.eye(Ktilde.shape[0], dtype=complex)
    return  (I + 1j * Ktilde) @ np.linalg.inv(I - 1j * Ktilde)

def getStildeFromInv(Ktilde_inv):
    """
    Compute the Stilde_inv matrix from the Ktilde_inv matrix.
    -(1 - i Ktilde_inv)(1 + i Ktilde_inv)^{-1}
    """
    I = np.eye(Ktilde_inv.shape[0], dtype=complex)
    return  (-I + 1j * Ktilde_inv) @ np.linalg.inv(I + 1j * Ktilde_inv)

def getOmega(mu, A):
    """
    Compute Omega(mu, A) = det(A)/(det(mu^2 + A A^dagger)^{1/2})
    """
    A_dagger = np.conjugate(A.T)
    det_A = np.linalg.det(A)
    det_term = np.linalg.det(sqrtm(mu**2 * np.eye(A.shape[0], dtype=complex) + A @ A_dagger))
    return det_A / det_term

def getQuantConds(Ktilde, B, Ktilde_inv, B_from_inv):
    """
    Compute the quantization conditions for the given Ktilde, B, and Ktilde_inv matrices.
    """
    Stilde = getStilde(Ktilde)
    StildeFromInv = getStildeFromInv(Ktilde_inv)
    CB = getCB(B)
    CBfromInv = getCB(B_from_inv)
    mu = 5.0

    qc_mats = [np.eye(Stilde.shape[0], dtype=complex) + Stilde@CB,
               np.eye(Stilde.shape[0], dtype=complex) - Ktilde@B]
    qc_mats_inv = [np.conj(StildeFromInv).T + CBfromInv,
                   Ktilde_inv - B_from_inv]

    qc_evs = [sorted(list(np.linalg.eigvals(mat)), key=lambda x: np.real(x)) for mat in qc_mats]
    qc_evs_inv = [sorted(list(np.linalg.eigvals(mat)), key=lambda x: np.real(x)) for mat in qc_mats_inv]
    qc_omegas = [getOmega(mu, mat) for mat in [qc_mats[0], qc_mats_inv[0], qc_mats[1], qc_mats_inv[1]]]
    return qc_evs, qc_evs_inv, qc_omegas

In [371]:
# set pwd to the directory of this script
script_dir = "/home/john/Documents/LatticeQCD/KBfit/source_testing/test_quant_conds"
os.chdir(script_dir)
# load in the arrays
B = np.load('B.npy')
B_from_inv = np.load('B_from_inv.npy')
Ktilde = np.load('Ktilde.npy')
Ktilde_inv = np.load('Ktilde_inv.npy')
np.set_printoptions(precision=8)

# get their Cayley transforms
CB = getCB(B)
Stilde = getStilde(Ktilde)
Stilde_inv = getStildeFromInv(Ktilde_inv)

In [372]:
qc_evs, qc_evs_inv, qc_omegas = getQuantConds(Ktilde, B, Ktilde_inv, B_from_inv)
qc_omegas
qc_evs = [sorted(list(evs), key=lambda x: np.real(x)) for evs in qc_evs]
qc_evs = np.array(qc_evs)


In [373]:
qc_omegas

[(9.213918378212943e-06+5.060875269296043e-05j),
 (-1.2969097546671993e-13-2.4267873840457708e-14j),
 (0.3785676343281114+0j),
 (-0.702487142542966+0j)]

In [374]:
qc_evs_inv

[[(-0.02908045373321034-0.23880415280408485j),
  (-0.028987718711938452-0.24123010858559443j),
  (-6.190354360842534e-05-0.006570393748999051j),
  (5.4914108473932524e-05-0.03000343115832579j),
  (0.0016467782750596207-0.10262300562148081j),
  (0.0018249887716535076-0.08320584122896078j),
  (0.44678284078014713+0.8232558448867525j),
  (1.4517907820473988-0.8921674115958211j),
  (1.8780384875785594-0.24012680294340139j)],
 [(-2569.5875115982617+0j),
  (-164.55817249160165+0j),
  (-6.330419492752181+0j),
  (10.245422230323312+0j),
  (70.25228033214157+0j),
  (130.09075202106646+0j),
  (276.64839615410176+0j),
  (763.0583104719979+0j),
  (2320.7634184721096+0j)]]

In [375]:
B.shape

(8, 8)